# This notebook performs some rudimentary Network Signal Strength Analytics. 

It displays the signal strengths in a graded color on a map. 

It uses data from an iOS app called RF Checker. This creates a .csv file of the signal strength of the RF antennae as you walk about


First we install some libraries
The beginnings of this notebook came from [an Article by George T. Silva](https://georgetsilva.github.io/posts/mapping-points-with-folium/)

In [25]:
import pandas as pd
import requests

[Folium](https://github.com/python-visualization/folium) is an interface to the [leaflet.org](http://leaflet.org) (also see [leafletjs](http://leafletjs.com/examples.html)) mapping API. It is used to help plot point on a map

DSX does not by default have Folium installed.

In [26]:
import sys
try:
    import folium
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install folium 
    else:
        !pip install --user folium
    import folium

## Information on the Data that will be loaded in
Prior to using this notebook, DataConnect on Bluemix was used to create a DashDB (now known as DB2 Warehouse on the Cloud) database. This database contains a the data from the RF Checker iOS app with some (extraneous) data removed, and a few columns renamed.

The following script was created automatically by the DSX connections interface

In [27]:
# The code was removed by DSX for sharing.

,Time,Signal Strength,Gps_Latitude,Gps_Longitude,Gps_Height,Gps_Horizontal_Accuracy,Gps_Vertical_Accuracy,Gps_Speed
0,2017-08-31 12:14:17,-85.0,39.698617,-104.957078,1633.825029,10.0,3.0,NaN
1,2017-08-31 12:14:18,-85.0,39.698624,-104.957060,1633.988115,10.0,3.0,1.09
2,2017-08-31 12:14:19,-85.0,39.698628,-104.957044,1633.839434,10.0,3.0,1.16
3,2017-08-31 12:14:20,-85.0,39.698634,-104.957032,1633.756670,10.0,3.0,1.09
4,2017-08-31 12:14:21,-85.0,39.698640,-104.957022,1633.702959,10.0,3.0,0.88


We only need for this notebook 4 of the fields from the DataFRame, i.e., 'Time', 'Signal Strength', 'Gps_Latitude', 'Gps_Longitude'

In [28]:
# extract the specific data from DashDB
signalCoordinates = data_df_1[['Time', 'Signal Strength', 'Gps_Latitude', 'Gps_Longitude']]

# Some general constants that are used later to access the 4 elements in the array of arrays
timeIndex = 0
strengthIndex = 1
latIndex = 2
longIndex = 3

# create a list
unfilteredLocationList = signalCoordinates.values.tolist()

When we display the map we need to position it at some coordinates. This script determines the lat and long of the middle of the map

In [29]:
# Used to find the center of the map
lengthOfList = float(len(unfilteredLocationList))
# find average lat
coords = [ item[latIndex] for item in unfilteredLocationList ]
lat = sum(coords) / lengthOfList
# find average Long
coords = [ item[longIndex] for item in unfilteredLocationList ]
long = sum(coords) / lengthOfList
# define where the map centeres itself 
centerOfMap = [lat, long]

We now need to create some GPS Elements. Each element represents the signal strength, is a collection of coordinates that have that signal, and finally a count of the \# of coordinates 

This routine cheats a little as it assumes the database list that is sorted by time. RF Checker logs a row every second, but every seecond is not necessarily a new RF strength. Therefore we group all Signal Strengths in time order, collapsing the all the points into that elment

In [30]:
# GPSElement = {
#   'Strength' : aDBmValue, 
#   'Coordinates' : [[aLatValue , aLongValue]...], 
#   'SignalCount' : aNumber }

GPSElementList = []  # this will contain the list of GPSElements to display on the map

# creates a GPSElement from an unfiltered DashDB row
def createGPSElement(anUnfilteredRow):
    coord = []
    coord.append([ anUnfilteredRow[latIndex], anUnfilteredRow[longIndex]]) 
    return {
        'Strength' : anUnfilteredRow[strengthIndex],
        'Coordinates' : coord,
        'SignalCount' : 0
    }

# Appends the coordinates of the unfileteredRow to the GPSElement
def addGPSCoordinate(aGPSElement, anUnfilteredRow):
    aGPSElement['Coordinates'].append([anUnfilteredRow[latIndex], anUnfilteredRow[longIndex]])
    aGPSElement['SignalCount'] = aGPSElement['SignalCount'] + 1
    return aGPSElement

# Assuming the unfilteredlist is sorted in time order when the signal strength changes we create a GPSElement
# other wise we simply append the [lat, long] to the current GPSElement record
for x in range(0, len(unfilteredLocationList)):
    unfilteredRow = unfilteredLocationList[x]
    aSignalStrength = unfilteredRow[strengthIndex]
    # Ignore values in the database that did not have a Signal Strength value
    if math.isnan(aSignalStrength) :
        continue
        
    if len(GPSElementList) == 0 or aSignalStrength != latestGPSElement['Strength']:
        latestGPSElement = createGPSElement(unfilteredRow)
        GPSElementList.append(latestGPSElement)
    else:
        addGPSCoordinate(latestGPSElement,unfilteredRow)

# Displaying the Map with graded colors represeting signal strength

Need a color scale for the RF Signal Strength. Use [Branca](https://github.com/python-visualization/branca)

In [31]:
import branca.colormap as cm
# find lowest signal Strength
low = min([ item['Strength'] for item in GPSElementList])
# find highest signal Strength
high = max([ item['Strength'] for item in GPSElementList])
colorScale = cm.linear.PuOr.scale(low, high)

At this point we are ready to create the map and plot the color graded line for the signal strengths

Ths script plots the route taken by RF Checker and color grades it

In [37]:
# create a map, center it on one of the coordinates, and set zoom level
map = folium.Map(location=centerOfMap, zoom_start=18)

for row in range(0, len(GPSElementList)):
    signalStrength = GPSElementList[row]['Strength']
    coordinates = GPSElementList[row]['Coordinates']
    signalStrengthCount = GPSElementList[row]['SignalCount']
    # display a polygon withthe coordinates
    folium.PolyLine(coordinates,
                    color=colorScale(signalStrength), 
                    weight=20, 
                    opacity=0.8).add_to(map)  
map

The below script prints the map using circles